## モデルの読み込み

In [1]:
from datasets import load_dataset
from torch.optim import AdamW
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer
)
import torch

model_id = "meta-llama/Llama-3.2-1B"


model = AutoModelForCausalLM.from_pretrained(
            model_id,device_map="cuda")

ref_model = AutoModelForCausalLM.from_pretrained(
            model_id,device_map="cuda")

tokenizer = AutoTokenizer.from_pretrained(model_id)


## データセットの読み込みと形状の確認

In [2]:
ds = load_dataset("HumanLLMs/Human-Like-DPO-Dataset")["train"][0]

In [3]:
chosen_text=ds["prompt"]+ds["chosen"]
rejected_text = ds["prompt"]+ds["rejected"]

In [4]:
chosen_tokens = tokenizer(chosen_text,return_tensors="pt")

chosen_input_ids= chosen_tokens["input_ids"].to(model.device)
chosen_attention_mask = chosen_tokens["attention_mask"].to(model.device)

In [5]:
outputs=model(input_ids=chosen_input_ids,attention_mask=chosen_attention_mask)

In [6]:
print("入力の形状",chosen_input_ids.shape)

# logit -> batch,seq,vocab
print("ロジットの形状",outputs.logits.shape)

入力の形状 torch.Size([1, 54])
ロジットの形状 torch.Size([1, 54, 128256])


In [7]:
# 時刻t=1におけるロジット
outputs.logits[:,1,:].shape

torch.Size([1, 128256])

In [8]:
# 時刻t=3におけるロジット
outputs.logits[:,3,:].shape

torch.Size([1, 128256])

## 確率の計算

In [9]:
import torch.nn.functional as f
# ロジット
logits = outputs.logits
logits

tensor([[[ 7.0543,  9.0268, 13.3232,  ..., -3.7595, -3.7596, -3.7596],
         [17.0245,  8.0694,  8.2875,  ..., -0.3987, -0.3988, -0.3994],
         [ 8.8644,  3.6967,  5.0680,  ...,  0.1932,  0.1933,  0.1927],
         ...,
         [-0.8128, -1.8097, -0.0361,  ...,  7.4711,  7.4714,  7.4720],
         [ 2.9312,  3.1062,  1.9763,  ...,  0.0846,  0.0849,  0.0849],
         [15.0278, 13.0997, 13.9905,  ..., -1.9659, -1.9658, -1.9656]]],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>)

In [10]:
prob=f.log_softmax(logits, dim=-1)
prob.shape

# dim=-1 は 最後の次元である vocab_size に沿ってlog_softmaxを適応 

torch.Size([1, 54, 128256])

In [11]:
# もしsoftmax関数を適応したら

prob_softmax=f.softmax(logits, dim=-1)

prob_softmax[0,1,:].sum()

#和が1になっていることを確認できる

tensor(1.0000, device='cuda:0', grad_fn=<SumBackward0>)

In [39]:
# それぞれの時刻の確率を求める
selected_log_probs = torch.gather(
    prob, dim=-1, index=chosen_input_ids.unsqueeze(-1)  # (batch_size, seq_len, 1)
)
# 余計な次元を除去
selected_log_probs = selected_log_probs.squeeze(-1)

# logから確率の積が和になるため sumを利用

log_p=selected_log_probs.sum(-1)

In [41]:
log_p

tensor([-576.8967], device='cuda:0', grad_fn=<SumBackward1>)

In [32]:
print(chosen_input_ids.shape)
#次元の追加
print(chosen_input_ids.unsqueeze(-1).shape)

torch.Size([1, 54])
torch.Size([1, 54, 1])


In [35]:
# 時刻t=0では、tokenの番号が128000の確率を選択
chosen_input_ids.unsqueeze(-1)[0,0]

tensor([128000], device='cuda:0')

### torch.gatherの使い方


In [24]:
## torch.gatherの使い方

import torch

# 入力テンソル torch.Size([1, 2, 4])
input_tensor = torch.tensor([[[10, 20, 30, 40],
                              [50, 60, 70, 80]]])

# 選択したいインデックス torch.Size([1, 2, 2]) に形を合わせる
index_tensor = torch.tensor([[[2, 3], [0, 1]]])



# 最後の次元に沿ってインデックスを指定して値を取得
output = torch.gather(input_tensor, dim=-1, index=index_tensor)
print(output.sum(-1))
# 出力:
# tensor([[30+40],
#         [50+60]])


tensor([[ 70, 110]])


In [14]:
## このことからそれぞれの次元に対して対数の確率を計算
selected_log_probs.shape

torch.Size([1, 54, 1])

In [15]:
selected_log_probs[0,0,0]

tensor(-15.1831, device='cuda:0', grad_fn=<SelectBackward0>)